# KX Order Book Building for ICE® Data Accelerator API examples

This notebook demonstrates calling the FSI Library APIs via REST for the KX Order Book Building for ICE® Data Accelerator. You will need a running instance of kdb Insights Enterprise with the Accelerator deployed.

kdb Accelerators are packages for kdb products that address typical industry use cases. Using an Accelerator allows you to leapfrog the first phases of a project or Insights implementation, skipping common boilerplate or initial effort and jumping straight to value adding work. They are a customization-first approach to developing on kdb products, helping you learn and unlock the value of KX products faster.

You can read more about the Accelerator in the [ICE® Orderbook Introduction](https://code.kx.com/insights/accelerators/fsi/ice-ob-introduction.html).

---
### Imports, Constants, & Authentication

In [25]:
# Python library imports
import os
import requests
from getpass import getpass
import json
import numpy as np
import pandas as pd
import datetime
from datetime import timezone
from requests_oauth2client import OAuth2Client, OAuth2ClientCredentialsAuth
from getpass import getpass

You should edit the constants below to reflect your particular kdb Insights instance details. You will be prompted for a Client and Secret.

In [ ]:
# Constants - set these as appropriate
INSIGHTS_HOST   = '<HOST>' # Your Insights Enterprise hostname
INSIGHTS_REALM  = '<REALM>' # Your Insights realm, typically 'insights'
INSIGHTS_CLIENT = input("kdb Insights client: ")           # Your insights client (service account name)
INSIGHTS_SECRET = getpass("kdb Insights client secret: ")  # Your insights secret

In [27]:
# Authentication
oauth2client = OAuth2Client(
    token_endpoint='https://' + INSIGHTS_HOST + '/auth/realms/' + INSIGHTS_REALM + '/protocol/openid-connect/token',
    auth=(INSIGHTS_CLIENT, INSIGHTS_SECRET))
oauth2cc = OAuth2ClientCredentialsAuth(oauth2client, scope='roles')

In [28]:
# Start/end time for our query (1 hour)
startTS = (datetime.datetime.now(timezone.utc) - datetime.timedelta(hours=1)).strftime('%Y.%m.%dD%H:%M:%S')
endTS = (datetime.datetime.now(timezone.utc).strftime('%Y.%m.%dD%H:%M:%S'))

---
## Calling getTicks

The getTicks API is used for raw data extraction and preprocessing. It retrieves records from a database table using a specified dictionary of arguments. You can read more about getTicks in the [Accelerator documentation](https://code.kx.com/insights/accelerators/fsi/getTicks.html).

In [29]:
# Build arguments dictionary
args = {
  "table": "OrderbookSnapshots",
  "startTS": startTS,
  "endTS": endTS,
  "scope": {"assembly": "fsi-app-ice-orderbook"}
}

# Make request & extract header & payload
r = requests.post('https://' + INSIGHTS_HOST + '/servicegateway/fsi/getTicks', auth=oauth2cc, json=args)
header = json.loads(r.text)['header']
payload = json.loads(r.text)['payload']

It is best practice when calling the APIs is to always check the header for errors - [code.kx.com](https://code.kx.com) has more information on kdb Insights Enterprise [response codes and errors](https://code.kx.com/insights/api/database/api/codes.html#return-code), and also has an [Accelerators API Error Glossary](https://code.kx.com/insights/accelerators/fsi/error-glossary.html).

In [30]:
header

{'rcvTS': '2024-11-18T15:21:31.208000000',
 'corr': '37ebe096-5154-43be-a7dc-ad651585ae8d',
 'protocol': 'gw',
 'entID': None,
 'logCorr': '37ebe096-5154-43be-a7dc-ad651585ae8d',
 'client': ':10.37.151.164:5050',
 'http': 'json',
 'api': '.fsi.getTicks',
 'userName': 'aj-client',
 'userID': 'e77d4a5e-1b7f-45dc-ad3d-38b2e213c6f6',
 'retryCount': 0,
 'to': '2024-11-18T15:22:31.208000000',
 'agg': ':10.37.152.93:5070',
 'pvVer': 4,
 'refVintage': 738871813870029,
 'rpID': 0,
 'startTS': '2024-11-18T15:20:00.000000000',
 'endTS': '2024-11-18T15:21:24.000000000',
 'rc': 0,
 'ac': 0,
 'ai': ''}

In [31]:
df = pd.DataFrame(payload)
df.head()

,eventTimestamp,instrumentID,exchangeID,bidPrice,bidSize,bidCount,totalBidSize,askPrice,askSize,askCount,totalAskSize,srcSys
0,2024-11-18T14:53:07.775393799,E:RR.,628,[570.4],[986],[1],986,[],[],[],0,628
1,2024-11-18T14:53:07.856869811,E:RR.,628,[570.4],[986],[1],986,[],[],[],0,628
2,2024-11-18T14:53:07.856995016,BA,326,[161.03],[20],[1],20,[],[],[],0,326
3,2024-11-18T14:53:07.857141094,E:RR.,628,[570.4],[986],[1],986,[],[],[],0,628
4,2024-11-18T14:53:07.857202230,BA,326,[161.03],[20],[1],20,[],[],[],0,326


---

### Query Order Book Snapshots by instrument name

Using the `idList` argument.

In [32]:
# Build arguments dictionary
args = {
  "table": "OrderbookSnapshots",
  "startTS": startTS,
  "endTS": endTS,
  "idList": "BA",
  "scope": {"assembly": "fsi-app-ice-orderbook"}
}

# Make request & extract header & payload
r = requests.post('https://' + INSIGHTS_HOST + '/servicegateway/fsi/getTicks', auth=oauth2cc, json=args)
header = json.loads(r.text)['header']
payload = json.loads(r.text)['payload']

In [33]:
df = pd.DataFrame(payload)
df.tail()

,eventTimestamp,instrumentID,exchangeID,bidPrice,bidSize,bidCount,totalBidSize,askPrice,askSize,askCount,totalAskSize,srcSys
28132,2024-11-18T15:21:22.534116510,BA,326,"[161.3, 161.2, 161.18]","[200, 8, 19]","[2, 1, 1]",227,"[161.42, 161.44, 161.46, 161.49, 161.5, 161.51...","[40, 1, 20, 49, 153, 28, 108, 7, 9, 300]","[1, 1, 1, 3, 9, 3, 2, 1, 1, 1]",11593,326
28133,2024-11-18T15:21:22.534257699,BA,326,"[161.3, 161.2, 161.18]","[202, 8, 19]","[3, 1, 1]",229,"[161.42, 161.44, 161.46, 161.49, 161.5, 161.51...","[40, 1, 20, 49, 153, 28, 108, 7, 9, 300]","[1, 1, 1, 3, 9, 3, 2, 1, 1, 1]",11593,326
28134,2024-11-18T15:21:23.551370557,BA,326,"[161.35, 161.3, 161.2, 161.18]","[24, 202, 8, 19]","[1, 3, 1, 1]",253,"[161.42, 161.44, 161.46, 161.49, 161.5, 161.51...","[40, 1, 20, 49, 153, 28, 108, 7, 9, 300]","[1, 1, 1, 3, 9, 3, 2, 1, 1, 1]",11593,326
28135,2024-11-18T15:21:23.551603186,BA,326,[],[],[],0,"[161.42, 161.44, 161.46, 161.49, 161.5, 161.51...","[40, 1, 20, 49, 153, 28, 108, 7, 9, 300]","[1, 1, 1, 3, 9, 3, 2, 1, 1, 1]",11593,326
28136,2024-11-18T15:21:23.551678924,BA,326,[161.3],[100],[1],100,"[161.42, 161.44, 161.46, 161.49, 161.5, 161.51...","[40, 1, 20, 49, 153, 28, 108, 7, 9, 300]","[1, 1, 1, 3, 9, 3, 2, 1, 1, 1]",11593,326


---

### Orderbook

The `orderbook` argument in `getTicks` is used for orderbook-specific logic.
Currently the only accepted input is  `plotOrderbookSnapshots`
This is used primarily to more easily plot OrderbookSnapshot data in a kdb IE view, or other visualization software.

Behaviour:
Filters the OrderbookSnapshots table for the max row within the specified time range.

Using that snapshotd data, the following values are calculated:

* `bestAsk` = min askPrice
* `bestBid` = max bidPrice
* `maxSize` = max (bidSize;askSize)
* `spread` = bestAsk - bestBid
* `midPoint` = (bestBid + bestAsk) / 2

In [34]:
# Build arguments dictionary
args = {
  "table": "OrderbookSnapshots",
  "startTS": startTS,
  "endTS": endTS,
  "orderbook": "plotOrderbookSnapshots",
  "scope": {"assembly": "fsi-app-ice-orderbook"}
}

# Make request & extract header & payload
r = requests.post('https://' + INSIGHTS_HOST + '/servicegateway/fsi/getTicks', auth=oauth2cc, json=args)
header = json.loads(r.text)['header']
payload = json.loads(r.text)['payload']

In [35]:
df = pd.DataFrame(payload)
df.tail()

,eventTimestamp,instrumentID,exchangeID,bidPrice,bidSize,bidCount,totalBidSize,askPrice,askSize,askCount,totalAskSize,srcSys,bestBid,bestAsk,maxSize,spread,midPoint,midPointList
0,2024-11-18T15:21:23.551678924,BA,326,[161.3],[100],[1],100,"[161.42, 161.44, 161.46, 161.49, 161.5, 161.51...","[40, 1, 20, 49, 153, 28, 108, 7, 9, 300]","[1, 1, 1, 3, 9, 3, 2, 1, 1, 1]",11593,326,161.3,161.42,300,0.12,161.36,[161.36]


---

### Passing different arguments
You can find a complete list of arguments accepted by `getTicks()` in our [documentation](https://code.kx.com/insights/1.11/accelerators/fsi/getTicks.html#arguments).

In [38]:
# Build arguments dictionary
args = {
  "table": "OrderbookSnapshots",
  "startTS": startTS,
  "endTS": endTS,
  "idList": "BA",
  "orderbook": "plotOrderbookSnapshots",
  "scope": {"assembly": "fsi-app-ice-orderbook"}
}

# Make request & extract header & payload
r = requests.post('https://' + INSIGHTS_HOST + '/servicegateway/fsi/getTicks', auth=oauth2cc, json=args)
header = json.loads(r.text)['header']
payload = json.loads(r.text)['payload']

In [39]:
df = pd.DataFrame(payload)
df.head()

,eventTimestamp,instrumentID,exchangeID,bidPrice,bidSize,bidCount,totalBidSize,askPrice,askSize,askCount,totalAskSize,srcSys,bestBid,bestAsk,maxSize,spread,midPoint,midPointList
0,2024-11-18T15:21:23.551678924,BA,326,[161.3],[100],[1],100,"[161.42, 161.44, 161.46, 161.49, 161.5, 161.51...","[40, 1, 20, 49, 153, 28, 108, 7, 9, 300]","[1, 1, 1, 3, 9, 3, 2, 1, 1, 1]",11593,326,161.3,161.42,300,0.12,161.36,[161.36]
